In [1]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer

In [2]:
name = "arjunth2001/priv_ftc"
model = AutoModelForSequenceClassification.from_pretrained(name)
tokenizer = AutoTokenizer.from_pretrained(name)

In [3]:
pipe = pipeline('text-classification', model=model, tokenizer=tokenizer,device=0)

In [4]:
labels=['First Party Collection/Use',
          'Third Party Sharing/Collection',
          'Other',
          'International and Specific Audiences',
          'Data Security',
          'User Choice/Control',
          'User Access, Edit and Deletion',
          'Data Retention',
          'Policy Change',
          'Do Not Track']
counter ={l:0 for l in labels}
def get_preds(x):
    global counter
    try:
        x =eval(x)
        preds = list(set(i["label"] for i in pipe(x)))
        for p in preds:
            counter[p]+=1
        return preds
    except:
        return []

In [5]:
import pandas as pd

In [6]:
df = pd.read_csv("fortune500_policies.csv")

In [7]:
from tqdm.auto import tqdm
tqdm.pandas()
df["preds"]=df["body"].progress_apply(lambda x: get_preds(x))

  0%|          | 0/485 [00:00<?, ?it/s]

/home2/arjunth2001/miniconda3/envs/op/lib/python3.7/site-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Token indices sequence length is longer than the specified maximum sequence length for this model (986 > 512). Running this sequence through the model will result in indexing errors
/home2/arjunth2001/miniconda3/envs/op/lib/python3.7/site-packages/transformers/pipelines/base.py:901: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,
Token indices sequence length is longer than the specified maximum sequence length for this mode

In [8]:
import json
with open("fortune500.json","w") as f:
    json.dump(counter,f,indent=4)

In [9]:
df.to_csv("fortune500_policies.csv",index=False)